In [37]:
import numpy as np
import scipy

class LBM_2_Carlemann:
    def __init__(self, N_grid, h_init, u_init):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1  # Speed of sound for D1Q3 lattice...is this the delta_x\delta_t ??
        self.g = 9.81  # Acceleration due to gravity (m/s^2)
        self.kn = 0.1 #Knudsen number, much less than 1 for chapman-enskogg expansion

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = N_grid # Number of grid points...my code for the F matrices makes the kernel die if this number is 81 or higher
        self.L = 0.0001  # Length of the domain (in meters)
        self.delta_t = self.L/self.Nx

        # Initialize macroscopic variables: density(height) and velocity field
        self.h = h_init  # height field
        self.u = u_init  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

    def get_F_single(self):

        g=self.g
        c=self.c_s

        F1 = 1/(self.tau*self.kn) * np.array([
        [0, 1, 1 ],
        [0, (1 / (2 * c)) - 1, -(1 / (2 * c))],
        [0, -1 / (2 * c), -1*(1 - (1 / (2 * c)))]
        ])
    

        F2 = 1/(self.tau*self.kn*2*c**2)* np.array([
        [-g, -g, -g, -g , -g - 4, -g + 4, -g, -g + 4, -g - 4],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2]
        ])

        F3 =np.array([
        [0, 0, 0, 0 , 2, -2, 0, -2, 2],
        [0, 0, 0, 0, -1, 1, 0, 1, -1],
        [0, 0, 0, 0, -1, 1, 0, 1, -1]
        ])

        F3 =  1/(self.tau*self.kn*2*c**2)*np.hstack([F3,F3,F3])

        return F1,F2,F3
    
    def create_one_hot_vectors(self,l):
       
        vector = np.eye(l)
        return vector


    def get_F_multiple(self,F1,F2,F3):

        F1_multiple_stacked_alpha = []
        F2_multiple_stacked_alpha = []
        F3_multiple_stacked_alpha = []
        
        delta = self.create_one_hot_vectors(self.Nx)

        for alpha in range(0,self.Nx):
        
            F1_multiple_stacked_alpha.append(np.kron(delta[alpha,:],F1)) #(33)
            F2_multiple_stacked_alpha.append(np.kron(delta[alpha,:],np.kron(delta[alpha,:],F2)))
            F3_multiple_stacked_alpha.append(np.kron(delta[alpha,:],np.kron(delta[alpha,:],np.kron(delta[alpha,:],F3))))

        # Stack matrices along the first dimension
        F1_multiple_stacked = np.vstack( F1_multiple_stacked_alpha)  #why vstack? in (33) looks like stack on columns
        F2_multiple_stacked = np.vstack( F2_multiple_stacked_alpha)  
        F3_multiple_stacked = np.vstack( F3_multiple_stacked_alpha)  

        return F1_multiple_stacked,F2_multiple_stacked,F3_multiple_stacked


    def get_A(self,F1,F2,F3):

        A11=F1
        A12=F2
        A13=F3

        I1 = np.eye(F1.shape[0])
        I2= np.eye(F2.shape[0])

        A22 = np.kron(I1,F1) + np.kron(F1,I1)
        A23 = np.kron(I2,F2) + np.kron(F2,I2)
        A33 = np.kron(I1,np.kron(I1,F1)) + np.kron(I1,np.kron(F1,I1))+ np.kron(F1,np.kron(I1,I1)) 

        return A11,A12,A13,A22,A23,A33
    
        

    def get_collision(self,A11,A12,A13,A22,A23,A33):

        zero1 = np.zeros((A22.shape[0], A11.shape[1]))  # Zero block for row 2
        zero2 = np.zeros((A33.shape[0], A11.shape[1] + A22.shape[1]))  # Zero block for row 3

        # Stack rows together
        row1 = np.hstack([A11, A12, A13])  # First row
        row2 = np.hstack([zero1, A22, A23])  # Second row
        row3 = np.hstack([zero2, A33])  # Third row
            
        # Stack all rows vertically
        C_collision = np.vstack([row1, row2, row3])

        return C_collision
    
    def get_S(self):

        main_diagonal = [1] * (self.Nx * len(self.e))
        #upper_diagonal = [-1] * ((self.Nx) * len(self.e)-3)
        lower_diagonal = [-1] * ((self.Nx) * len(self.e)-3)

        # Create the matrix
        S =  np.diag(main_diagonal, k=0)     # Main diagonal
        #S += np.diag(upper_diagonal, k=3)    # Upper diagonal 
        S += np.diag(lower_diagonal, k=-3)   # Lower diagonal 
       
        #add periodic BC
        S[0,-3]=S[1,-2]=S[2,-1]=-1
        #S[-3,0]=S[-2,1]=S[-1,2]=1
        
        #multiply with e_m
        S[::3, :] = 0 
        S[1::3, :] *= 1 
        S[2::3, :] *= -1 

        S = (1/self.delta_t)* S

        return S


    def get_B(self,S1):

        I1 = np.eye(S1.shape[0])

        B11=S1
        B22 = np.kron(I1,S1) + np.kron(S1,I1)
        B33 = np.kron(I1,np.kron(I1,S1)) + np.kron(S1,np.kron(I1,I1)) + np.kron(I1,np.kron(S1,I1))

        return B11,B22,B33
        
    
    def get_streaming_first_ord(self, B11, B22, B33):

        C_streaming = scipy.linalg.block_diag(B11, B22, B33)

        return C_streaming
      
    
    def gen_streaming_sec_ord(self):
        
        Q= len(self.e)
        n = self.Nx
        inv_delta = n/(2*self.L)
        dim = n*Q
        I = np.identity(dim)
        S = np.zeros((dim, dim))
        
        for i in range(dim):
            #deal with edge case here...periodic or bounce back BC...here I do code for periodic
            if i<Q:
                S[i,i+Q] = inv_delta*self.e[(i%3)]
                S[i, (dim-Q)+i] = -inv_delta*self.e[(i%3)]
            elif i>dim -Q - 1:
                S[i,dim-i] = inv_delta*self.e[(i%3)]
                S[i,i-Q] = -inv_delta*self.e[(i%3)]
            else:
                S[i,i+Q] = inv_delta*self.e[(i%3)]
                S[i,i-Q] = -inv_delta*self.e[(i%3)]


        B11 = S
        B22 = np.kron(S,I) + np.kron(I, S)
        B33 = np.kron(np.kron(S,I), I) + np.kron(np.kron(I,S), I) + np.kron(np.kron(I,I), S)

        C1 = np.vstack((B11,np.zeros((B22.shape[0]+B33.shape[0],B11.shape[1]))))
        C2 = np.vstack((np.zeros((B11.shape[0],B22.shape[1])),B22,np.zeros((B33.shape[0],B22.shape[1]))))
        C3 = np.vstack((np.zeros((B11.shape[0]+B22.shape[0],B33.shape[1])), B33))

        Cs = np.hstack((C1,C2,C3))

        return Cs
        
    


In [38]:
N_grid = 4
h_init = [0.8,1,1,1]
u_init = [0,0,0,0]

Gen = LBM_2_Carlemann(N_grid, h_init, u_init )

F1_single,F2_single,F3_single  = Gen.get_F_single()

F1_,F2_,F3_ = Gen.get_F_multiple(F1_single,F2_single,F3_single)

A11_, A12_, A13_, A22_, A23_, A33_ = Gen.get_A(F1_,F2_,F3_)

C_collision = Gen.get_collision(A11_, A12_, A13_, A22_, A23_, A33_)

###################################################################################################

S_ = Gen.get_S()

B11_,B22_,B33_= Gen.get_B(S_)

C_streaming_first = Gen.get_streaming_first_ord(B11_, B22_, B33_)

C_streaming_sec = Gen.gen_streaming_sec_ord()

CL_LBE_Matrix = C_collision + C_streaming_first
#CL_LBE_Matrix = C_collision + C_streaming_sec



In [39]:
def embed_matrix(C, delta_t, num_steps):
    """
    Embeds a matrix C into a larger matrix A with specified properties.

    Parameters:
        C (np.ndarray): The matrix to embed (must be square).
        delta_t (float): The length of each time step.
        num_steps (int): The number of time steps (and thus of cascaded blocks in A).

    Returns:
        np.ndarray: The constructed matrix A.
    """
    # Validate inputs
    if not (isinstance(C, np.ndarray) and C.ndim == 2 and C.shape[0] == C.shape[1]):
        raise ValueError("C must be a square matrix.")

    # Identity matrix with the same size as C
    Id = np.eye(C.shape[0])

    # Compute -O = -(Id + delta_t * C)
    O = -(Id + delta_t * C)

    # Size of the large matrix A
    A_size = num_steps * C.shape[0]

    # Initialize A as a zero matrix
    A = np.zeros((A_size, A_size))

    # Fill in the diagonal blocks
    for i in range(num_steps):
        # Main diagonal (Identity blocks)
        start_idx = i * C.shape[0]
        A[start_idx:start_idx + C.shape[0], start_idx:start_idx + C.shape[0]] = Id

        # Secondary diagonal (-O blocks)
        if i > 0:
            prev_idx = (i - 1) * C.shape[0]
            A[start_idx:start_idx + C.shape[0], prev_idx:prev_idx + C.shape[0]] = O

    return A

In [40]:
N_time = 4  # Number of time steps
delta_t = Gen.delta_t
Lin_Euler_Matrix = embed_matrix(CL_LBE_Matrix, delta_t, N_time)
print("Shape of the Matrix to invert:", Lin_Euler_Matrix.shape)

Shape of the Matrix to invert: (7536, 7536)


In [41]:
def initial_distribution( Nx,h,u):

    #if velocities are null everywhere
    #initial_distribution = np.kron(Gen.h,(1/6,2/3,1/6))

    distr = np.zeros((Nx,3))

    for g in range (Nx):
        for i in range(3):
            if i==0:
                distr[g,i]= h[g]*1/6 - u[g] 
            if i==1:
                distr[g,i]= h[g]*2/3
            if i==2:
                distr[g,i]= h[g]*1/6 + u[g] 
            
    initial_distribution = np.hstack([distr[g,:] for g in range (Gen.Nx)])  # Function of 3 values in the 1D case
    
    return initial_distribution


def append_zeros(f, N):
    # Ensure `f` is a NumPy array
    f = np.array(f)

    # Create N zero vectors of the same shape as `f`
    zero_vector = np.zeros_like(f)
    zeros_to_append = np.tile(zero_vector, (N-1,))

    # Concatenate `f` with the appended zeros
    result = np.concatenate([f, zeros_to_append])

    return result

In [42]:
f1 = initial_distribution(Gen.Nx, Gen.h, Gen.u)
f2 = np.kron(f1,f1)
f3 = np.kron(f2,f1)

phi_t0 = np.hstack((f1,f2,f3))

phi = append_zeros(phi_t0, N_time)

In [43]:
#eigenvalues, _ = np.linalg.eig(Lin_Euler_Matrix)

#smallest_eigenvalue = np.min(eigenvalues)
#print(f"The smallest eigenvalue is: {smallest_eigenvalue}")

In [44]:
#cond_number = np.linalg.cond(Lin_Euler_Matrix)
#print(f"The condition number of the matrix is: {cond_number}")

In [45]:
Inverted_matrix = np.linalg.inv(Lin_Euler_Matrix)
x = np.dot(Inverted_matrix, phi)

In [46]:
def extract_phi(x, subvector_dim, N):
    if len(x) < N * subvector_dim:
        raise ValueError("The length of x is too small for the given N and subvector_dim.") 
    # Extract phi components
    phi = [x[i * subvector_dim : (i + 1) * subvector_dim] for i in range(N)]
    return phi

def phi_truncation(phi_list, N):
    num_values = 3 * N
    truncated_phi = [phi[:num_values] for phi in phi_list]
    return truncated_phi

def divide_truncated_phi(truncated_phi, N):
    result = []
    for phi in truncated_phi:
        # Ensure the truncated phi has at least 3 * N elements
        if len(phi) < 3 * N:
            raise ValueError("Each truncated phi must have at least 3 * N elements.")
        
        # Divide phi into N groups of 3
        groups = [phi[3 * i : 3 * (i + 1)] for i in range(N)]
        result.extend(groups)
    
    return result 


In [47]:
# x = np.arange(10845)  # Example: [0, 1, 2, ..., 10844]
N_grid = Gen.Nx

grid_evolution = np.zeros((N_grid,N_time,3))

subvector_dim = 3 * N_grid + 9 * N_grid**2 + 27 * N_grid**3
print("Subvector dimension:", subvector_dim)

# Extract phi components for each time step
phi = extract_phi(x, subvector_dim, N_time)

# Extract the first 3 * N values from each phi_i
truncated_phi = phi_truncation(phi, N_grid)

# Divide each truncated phi into N groups of 3 and combine them
three_dim_vectors = divide_truncated_phi(truncated_phi, N_grid)

# Display the results
for i, vec in enumerate(three_dim_vectors, start=0):
    print(f"Grid {i % Gen.Nx} at time { i // Gen.Nx}: {vec}")
    grid_evolution[i % Gen.Nx, i // Gen.Nx,:] = vec


Subvector dimension: 1884
Grid 0 at time 0: [0.13333333 0.53333333 0.13333333]
Grid 1 at time 0: [0.16666667 0.66666667 0.16666667]
Grid 2 at time 0: [0.16666667 0.66666667 0.16666667]
Grid 3 at time 0: [0.16666667 0.66666667 0.16666667]
Grid 0 at time 1: [0.13304753 0.4003718  0.16703847]
Grid 1 at time 1: [0.16522357 0.80155949 0.13489282]
Grid 2 at time 1: [0.16498761 0.66840544 0.16840544]
Grid 3 at time 1: [0.16627021 0.66717146 0.16717146]
Grid 0 at time 2: [0.1327195  0.13396488 0.16756419]
Grid 1 at time 2: [0.16360079 1.20454542 0.16883672]
Grid 2 at time 2: [0.16331332 0.53698163 0.13662305]
Grid 3 at time 2: [0.16584275 0.66646934 0.16893731]
Grid 0 at time 3: [ 0.13228602 -0.39809434  0.16938255]
Grid 1 at time 3: [0.16129564 2.27777867 0.1702169 ]
Grid 2 at time 3: [ 0.16214385 -0.12937559  0.17004328]
Grid 3 at time 3: [0.16534515 0.79655501 0.13722102]


In [48]:
#heigth
h = np.zeros((N_grid,N_time))

for t in range (N_time):
    print("Timestep", t)
    for g in range (N_grid):
        h[g, t] = sum(grid_evolution[g,t,:])
        print(f"Grid {g} height: {h[g,t]}")
    print("")
    



Timestep 0
Grid 0 height: 0.7999999999999995
Grid 1 height: 0.9999999999999996
Grid 2 height: 0.9999999999999994
Grid 3 height: 0.9999999999999994

Timestep 1
Grid 0 height: 0.7004577999999997
Grid 1 height: 1.1016758749999993
Grid 2 height: 1.0017984999999998
Grid 3 height: 1.0006131249999997

Timestep 2
Grid 0 height: 0.4342485766376427
Grid 1 height: 1.5369829265878199
Grid 2 height: 0.8369180072931457
Grid 3 height: 1.0012493952132797

Timestep 3
Grid 0 height: -0.09642576008458129
Grid 1 height: 2.609291215136822
Grid 2 height: 0.20281154137427013
Grid 3 height: 1.0991211806184054

